In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import glob
import matplotlib.image as mpimg
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
        vgg16,
        resnet50,
        inception_v3)
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers

# Load Data

In [ ]:
train_dir = '../input/cassava-leaf-disease-classification/train_images'
test_dir = '../input/cassava-leaf-disease-classification/test_images'
train_names = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv',dtype=str)

In [ ]:
print(train_names.head())
print(len(train_names))

# Plot the Data

First we will want to visualize our data

In [ ]:
plt.figure(figsize=(8,8), num=0)
for cntr, filename in enumerate(glob.iglob(train_dir + '**/*.jpg', recursive=True)):
    if cntr == 9:
        image = Image.open(filename)
        image_shape = np.array(image).shape
        print(f'The shape of the images is: {image_shape}')
        break
    elif cntr < 9:
        img = mpimg.imread(filename)
        plt.subplot(3,3,cntr+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img)
plt.show()

In [ ]:
batch_size = 256
img_height = image_shape[0]
img_width = image_shape[1]

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_names.head()

# Constructing the train generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.2)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
dataframe = train_names,
directory = train_dir,
x_col = "image_id",
y_col = "label",
subset = "training",
batch_size = batch_size,
seed = 42,
shuffle = True,
class_mode = "sparse",
target_size = (img_height, img_width))

In [ ]:
valid_generator = train_datagen.flow_from_dataframe(
dataframe = train_names,
directory = train_dir,
x_col = "image_id",
y_col = "label",
subset = "validation",
batch_size = batch_size,
seed = 42,
shuffle = True,
class_mode = "sparse",
target_size = (img_height, img_width))

# Constructing the test generator

In [ ]:
# Will be given to us at the end...

# Iterating over our Data and plotting it
## Now the data and labels are together!

In [ ]:
image_batch, labels_batch = next(iter(train_generator))

In [ ]:
labels_batch

In [ ]:
plt.figure(figsize=(14,14), num=1)
for i in range(32):
    
#     max_value = max(labels_batch[i])
#     labels_list = list(labels_batch[i])
#     max_index = labels_list.index(max_value)
    max_index = labels_batch[i]
    plt.subplot(8,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(image_batch[i])
    plt.title(max_index)
plt.show()

In [ ]:
image_batch, labels_batch = next(iter(train_generator))

In [ ]:
print(image_batch.shape)
print(labels_batch.shape)

In [ ]:
image_batch, labels_batch = next(iter(valid_generator))

In [ ]:
print(image_batch.shape)
print(labels_batch.shape)

In [ ]:
labels_batch

In [ ]:
plt.figure(figsize=(14,14), num=1)
for i in range(32):
    
#     max_value = max(labels_batch[i])
#     labels_list = list(labels_batch[i])
#     max_index = labels_list.index(max_value)
    max_index = labels_batch[i]

    plt.subplot(8,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(image_batch[i])
    plt.title(max_index)
plt.show()

# Building the Model

In [ ]:
def my_block(inputs, block_name='MyBlock'):
    if block_name == 'MyBlock':
        with tf.name_scope(block_name):
            conv = layers.Conv2D(filters=3, strides=2, kernel_size=2, padding='same')(inputs)
            bn = layers.BatchNormalization()(conv)
            relu = layers.ReLU()(bn)
            return relu
    
    elif block_name == 'MyEnd':
        with tf.name_scope(block_name):
            conv = layers.Conv2D(filters=3, strides=2, kernel_size=2, padding='same')(inputs)
            bn = layers.BatchNormalization()(conv)
            relu = layers.ReLU()(bn)
            return relu

In [ ]:
 def preprocessing_layer(inputs, block_name='MyPre'):
    with tf.name_scope(block_name):
        pre = preprocessing.Resizing(int(224*4), int(224*4), interpolation='bilinear')(inputs)
        pre = preprocessing.RandomFlip('horizontal')(pre)
        pre = preprocessing.RandomFlip('vertical')(pre)
        pre = preprocessing.RandomContrast(0.5)(pre) 
    return pre

In [ ]:
resnet_model = resnet50.ResNet50(weights='imagenet')
for layer in resnet_model.layers[:161]:
    layer.trainable = False

In [ ]:
inputs = keras.Input(shape=(img_height, img_width, 3))

In [ ]:
x = preprocessing_layer(inputs, block_name='MyPre')
x = my_block(x, block_name='MyBlock')
x = my_block(x, block_name='MyEnd')

# x = resnet_model(x)

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(100, activation='relu')(x)
outputs = layers.Dense(5, activation='relu')(x)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
train_samples = train_generator.samples
valid_samples = valid_generator.samples

In [ ]:
print(train_samples, valid_samples)

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch = np.ceil(train_samples / batch_size),
        epochs = 5,
        validation_data = valid_generator,
        validation_steps = np.ceil(valid_samples / batch_size))

In [ ]:
history_frame = pd.DataFrame(history.history)

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

# Predictions on the test Data

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)
test_dir = '../input/cassava-leaf-disease-classification/test_images'

In [ ]:
file_list = [i.split('/')[-1] for i in glob.glob(test_dir + "/*.jpg")]
labels = ['0' for i in range(len(file_list))]

In [ ]:
d = {'image_id': file_list, 'label': labels}
df = pd.DataFrame(data=d)

In [ ]:
train_names.label[0]

In [ ]:
df.label[0]

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
dataframe = df,
directory = test_dir,
x_col = "image_id",
y_col = "label",
subset = "training",
batch_size = 1,
seed = 42,
shuffle = True,
class_mode = "sparse",
target_size = (img_height, img_width))

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)

In [ ]:
predictions = model.predict(test_generator,steps = nb_samples)

In [ ]:
predictions = np.argmax(predictions, axis=1) 

In [ ]:
predictions

In [ ]:
df.label = predictions

# Exhibition of our results

In [ ]:
df

In [ ]:
plt.imshow(image_batch[0])

In [ ]:
df.to_csv('submission.csv', index=False)